In [1]:
import pandas as pd
import requests
import os

### Remove Unnecessary Columns ###

In [2]:
file_csv = '~/code/harlqeuinht/which_horse/raw_data/combined_flat2_csv.csv'

In [3]:
df = pd.read_csv(file_csv)

/tmp/ipykernel_134545/2113571882.py:1: DtypeWarning: Columns (22,53,62,71,80,89,98,107,116) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_csv)


In [4]:
df.head()

,meeting_id,meeting_name,date,country_code,event_number,distance,distance_unit,distance_furlongs,distance_raw_furlongs,race_class,...,run_7_dsr,run_8_raw_post_race_rating_int,run_8_raw_post_race_rating_symbol,run_8_final_rating_int,run_8_race_type,run_8_race_class,run_8_race_class_normalised,run_8_track_type,run_8_win_lose,run_8_dsr
0,3668,AYR,2020-07-20,GBR,1,1100,yards,5f,5.0,6,...,420.0,73.0,NaN,80.0,FLAT,5,Novice,TURF,0.0,429.0
1,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,304.0,63.0,+,74.0,FLAT,6,Handicap,AWT,1.0,317.0
2,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,395.0,67.0,NaN,79.0,FLAT,4,Handicap,TURF,0.0,397.0
3,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,327.0,67.0,?,77.0,FLAT,5,Handicap,TURF,0.0,339.0
4,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
columns_to_drop = df.filter(regex='[678]').columns
columns_to_drop = columns_to_drop.drop('bet365_odds')
df = df.drop(columns=(columns_to_drop))

Create a DataFrame which stores the meeting location name and long & lat. Note this has been done on the sample dataset, not the full dataset

### Locations API ###

In [6]:
location_names = sorted(df['meeting_name'].unique())
#location_names = [location for location in location_names if location.lower() != 'newmarket (july)']
locations_df = pd.DataFrame(location_names, columns=['meeting_name'])
locations_df['location_names_cleaned'] = locations_df['meeting_name'].replace({'BANGOR-ON-DEE':'BANGOR', 'NEWMARKET (JULY)':'NEWMARKET', ' ':'_'})
locations_df['location_names_cleaned'] = locations_df['location_names_cleaned'].str.replace(' ', '_')

Temporarily reduce the dataframe to 5 rows

In [7]:
locations_df.head()

,meeting_name,location_names_cleaned
0,AINTREE,AINTREE
1,ASCOT,ASCOT
2,AYR,AYR
3,BANGOR-ON-DEE,BANGOR
4,BATH,BATH


Create endpoint URLs for Geolocation API

In [8]:
api_urls = []
all_results = []
for index, location in enumerate(locations_df['location_names_cleaned']):
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    api_key = os.environ['KEY']
    params = f'address={location}+racecourse&components=country:GB&key={api_key}'
    endpoint = f'{base_url}{params}'
    results = requests.get(endpoint).json()
    # results_dict = {'location':location,
    #                 'co_ords':{'lat':results['results'][0]['geometry']['location']['lat'],
    #                            'lng':results['results'][0]['geometry']['location']['lng']
    #                           }
    #                }
    # all_results.append(results_dict)
    locations_df.loc[index, 'lat'] = results['results'][0]['geometry']['location']['lat']
    locations_df.loc[index, 'lng'] = results['results'][0]['geometry']['location']['lng']


In [15]:
locations_df.head(10)

,meeting_name,location_names_cleaned,lat,lng
0,AINTREE,AINTREE,53.473476,-2.954128
1,ASCOT,ASCOT,51.412167,-0.679485
2,AYR,AYR,55.467721,-4.612545
3,BANGOR-ON-DEE,BANGOR,53.004035,-2.909421
4,BATH,BATH,51.418670,-2.407209
5,BEVERLEY,BEVERLEY,53.843136,-0.458438
6,BRIGHTON,BRIGHTON,50.829821,-0.112602
7,CARLISLE,CARLISLE,54.858994,-2.928866
8,CATTERICK,CATTERICK,54.388054,-1.649181
9,CHELMSFORD,CHELMSFORD,51.842047,0.512814


### Weather API ###

In [26]:
#df = pd.merge(df,locations_df, on = 'meeting_name')
df.head()

,meeting_id,meeting_name,date,country_code,event_number,distance,distance_unit,distance_furlongs,distance_raw_furlongs,race_class,...,run_5_final_rating_int,run_5_race_type,run_5_race_class,run_5_race_class_normalised,run_5_track_type,run_5_win_lose,run_5_dsr,location_names_cleaned,lat,lng
0,3668,AYR,2020-07-20,GBR,1,1100,yards,5f,5.0,6,...,66.0,FLAT,5,Handicap,TURF,0.0,342.0,AYR,55.467721,-4.612545
1,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,71.0,FLAT,6,Handicap,TURF,0.0,279.0,AYR,55.467721,-4.612545
2,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,68.0,FLAT,3,Handicap,TURF,0.0,383.0,AYR,55.467721,-4.612545
3,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,69.0,FLAT,6,Handicap,TURF,0.0,272.0,AYR,55.467721,-4.612545
4,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AYR,55.467721,-4.612545


In [16]:
race_dates = df['date']
race_locations = df['meeting_name']
unique_race_days_df = pd.DataFrame({'race_dates': race_dates, 'race_locations': race_locations})
unique_race_days_df = unique_race_days_df.drop_duplicates()

In [19]:
unique_race_days_df

,race_dates,race_locations
0,2020-07-20,AYR
75,2020-07-20,MARKET RASEN
81,2020-07-20,WINDSOR
174,2020-07-19,NEWBURY
268,2020-07-19,YORK
...,...,...
281633,2023-09-30,RIPON
281694,2023-09-30,NEWMARKET
281794,2023-09-30,HAYDOCK
281848,2023-09-30,CHELMSFORD


In [20]:
unique_race_days_df = pd.merge(unique_race_days_df, locations_df, how='left', left_on='race_locations', right_on='meeting_name')


In [21]:
unique_race_days_df

,race_dates,race_locations,meeting_name,location_names_cleaned,lat,lng
0,2020-07-20,AYR,AYR,AYR,55.467721,-4.612545
1,2020-07-20,MARKET RASEN,MARKET RASEN,MARKET_RASEN,53.380460,-0.314614
2,2020-07-20,WINDSOR,WINDSOR,WINDSOR,51.485059,-0.633749
3,2020-07-19,NEWBURY,NEWBURY,NEWBURY,51.397169,-1.306517
4,2020-07-19,YORK,YORK,YORK,53.940811,-1.092174
...,...,...,...,...,...,...
5234,2023-09-30,RIPON,RIPON,RIPON,54.125176,-1.499693
5235,2023-09-30,NEWMARKET,NEWMARKET,NEWMARKET,52.238014,0.374746
5236,2023-09-30,HAYDOCK,HAYDOCK,HAYDOCK,53.480950,-2.627852
5237,2023-09-30,CHELMSFORD,CHELMSFORD,CHELMSFORD,51.842047,0.512814


In [23]:
unique_race_days_df.drop('race_locations', axis=1)

,race_dates,meeting_name,location_names_cleaned,lat,lng
0,2020-07-20,AYR,AYR,55.467721,-4.612545
1,2020-07-20,MARKET RASEN,MARKET_RASEN,53.380460,-0.314614
2,2020-07-20,WINDSOR,WINDSOR,51.485059,-0.633749
3,2020-07-19,NEWBURY,NEWBURY,51.397169,-1.306517
4,2020-07-19,YORK,YORK,53.940811,-1.092174
...,...,...,...,...,...
5234,2023-09-30,RIPON,RIPON,54.125176,-1.499693
5235,2023-09-30,NEWMARKET,NEWMARKET,52.238014,0.374746
5236,2023-09-30,HAYDOCK,HAYDOCK,53.480950,-2.627852
5237,2023-09-30,CHELMSFORD,CHELMSFORD,51.842047,0.512814


In [25]:
# code has been utilised in the dataframe above

# for date in race_dates:
#     for location in race_locations:
#         print(df[(df['date'] == date) & (df['meeting_name'] == loc_)])
#     break

Generate API endpoints for each unique location/date combo

In [68]:
def generate_endpoint(row):
    base_url = 'https://archive-api.open-meteo.com/v1/archive?'
    latitude = row['lat']
    longitude = row['lng']
    date = row['race_dates']
    params = '&daily=temperature_2m_mean,precipitation_sum,wind_speed_10m_max&wind_speed_unit=mph'
    return f'{base_url}&latitude={latitude}&longitude={longitude}&start_date={date}&end_date={date}&{params}'

unique_race_days_df['endpoint'] = unique_race_days_df.apply(generate_endpoint, axis=1)

Create a sample df with 5 rows and test the API call

In [74]:
test_df = unique_race_days_df.head()
test_df

,race_dates,race_locations,meeting_name,location_names_cleaned,lat,lng,endpoint
0,2020-07-20,AYR,AYR,AYR,55.467721,-4.612545,https://archive-api.open-meteo.com/v1/archive?...
1,2020-07-20,MARKET RASEN,MARKET RASEN,MARKET_RASEN,53.380460,-0.314614,https://archive-api.open-meteo.com/v1/archive?...
2,2020-07-20,WINDSOR,WINDSOR,WINDSOR,51.485059,-0.633749,https://archive-api.open-meteo.com/v1/archive?...
3,2020-07-19,NEWBURY,NEWBURY,NEWBURY,51.397169,-1.306517,https://archive-api.open-meteo.com/v1/archive?...
4,2020-07-19,YORK,YORK,YORK,53.940811,-1.092174,https://archive-api.open-meteo.com/v1/archive?...


In [81]:
def get_weather_data(row):
    response =requests.get(row['endpoint'])
    data = response.json()
    temp = data['daily']['temperature_2m_mean'][0]
    precipitation = data['daily']['precipitation_sum'][0]
    wind =data['daily']['wind_speed_10m_max'][0]
    return temp, precipitation, wind

test_df[['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max']] = test_df.apply(get_weather_data, axis=1, result_type='expand')

/tmp/ipykernel_134545/1069254005.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max']] = test_df.apply(get_weather_data, axis=1, result_type='expand')


In [82]:
test_df

,race_dates,race_locations,meeting_name,location_names_cleaned,lat,lng,endpoint,temperature_2m_mean,precipitation_sum,wind_speed_10m_max
0,2020-07-20,AYR,AYR,AYR,55.467721,-4.612545,https://archive-api.open-meteo.com/v1/archive?...,12.4,1.6,12.9
1,2020-07-20,MARKET RASEN,MARKET RASEN,MARKET_RASEN,53.380460,-0.314614,https://archive-api.open-meteo.com/v1/archive?...,14.0,0.0,11.2
2,2020-07-20,WINDSOR,WINDSOR,WINDSOR,51.485059,-0.633749,https://archive-api.open-meteo.com/v1/archive?...,15.9,0.0,11.3
3,2020-07-19,NEWBURY,NEWBURY,NEWBURY,51.397169,-1.306517,https://archive-api.open-meteo.com/v1/archive?...,16.4,4.3,10.7
4,2020-07-19,YORK,YORK,YORK,53.940811,-1.092174,https://archive-api.open-meteo.com/v1/archive?...,14.6,0.0,10.6


In [78]:
type(test_df['temperature_2m_mean'][0])

list